In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.mplot3d import Axes3D


def plot_currents():

    data = currents

    fig, ax = plt.subplots()
    im = ax.imshow(data)
    ax.set_aspect("auto")
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
             rotation_mode="anchor")

    ax.set_title("Network currents history")
    ax.set_xlabel("Neuron")
    ax.set_ylabel("Iteration")

    plt.tight_layout()
    plt.show()


def plot_weights():

    fig, ax = plt.subplots()
    im = ax.contourf(weights)
    ax.set_aspect("auto")

    ax.set_title("Weights matrix")
    ax.set_xlabel("Neuron $i$")
    ax.set_ylabel("Neuron $j$")

    plt.tight_layout()

    fig.colorbar(im, ax=ax)
    plt.show()


def plot_mean_weights(network):

    fig, ax = plt.subplots()
    im = ax.plot(weights_mean)
    # ax.set_aspect("auto")

    ax.set_title("Weights learning rule")
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Difference of means")

    # plt.tight_layout()

    plt.show()


def plot_p_recall(network):

    fig, ax = plt.subplots()
    im = ax.plot(p_recall_history)

    ax.set_title("Probability of recall")
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Pattern match")
    ax.set_ylim((-0.1, 1.1))

    plt.show()


def plot_energy(network):

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    x = np.arange(0, num_neurons, 1)
    y = np.arange(0, num_neurons, 1)
    x, y = np.meshgrid(x, y)
    z = np.copy(network.weights)

    for i in range(num_neurons):
        for j in range(num_neurons):
            z[i, j] *= currents[-1, j]

    surf = ax.plot_surface(x, y, z, alpha=0.9, cmap="viridis",
                           antialiased=True)

    ax.set_title("Energy landscape")
    ax.set_xlabel("Neuron $i$")
    ax.set_ylabel("Neuron $j$")
    ax.set_zlabel("Energy")

    plt.show()


In [9]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


num_neurons = 1000
p = 16
f = 0.1
inverted_fraction = 0.3
noise_variance = 65
first_p = 0
learning_rate = 0.3
forgetting_rate=0.1

assert learning_rate <= 1
assert forgetting_rate <= 1

weights = np.zeros((num_neurons, num_neurons))

next_theoretical_weights = np.zeros_like(weights)

next_weights = np.zeros_like(weights)
weights_mean = []

p_recall_history = []

active_fraction = f

initial_currents = np.zeros(num_neurons)

patterns = \
            np.random.choice([0, 1], p=[1 - f, f],
                             size=(p, num_neurons))
print("\nPatterns:\n", patterns)

currents = np.zeros((1, num_neurons), dtype=int)
patterns_evolution = None

question_pattern = np.zeros(num_neurons)

    # def present_pattern(self, item):
    #     kanji = item["kanji"]
    #     meaning = item["meaning"]
    #
    #     self.patterns.append(np.concatenate((kanji, meaning), axis=None))

def binarize_item(item):
    """
    Item number to binary and append zeros according to network size.

    :param item: int item index
    :return: bin_item binary vector
    """
    question_array = np.array([item])
    bin_item = ((question_array[:, None]
                     & (1 << np.arange(8))) > 0).astype(int)
    bin_item = np.append(bin_item, np.zeros(self.num_neurons
                                            - bin_item.size))

    print("Item given as pattern:", bin_item)
    return bin_item


def distort_pattern(pattern, proportion):
    """
    Inverts the array in random positions proportional to array size.

    :param pattern: array-like binary vector to distort
    :param proportion: float 0 to 1, 1 being full array inversion

    :return pattern: array-like binary vector with inverted elements
    """

    num_inversions = int(pattern.size * proportion)
    assert proportion != 1
    idx_reassignment = np.random.choice(pattern.size, num_inversions,
                                        replace=False)
    pattern[idx_reassignment] = np.invert(pattern[idx_reassignment] - 2)
    print("\nDistorted pattern (i.e. initial currents)...\n", pattern,
          "\n ...in positions\n", idx_reassignment)
    return pattern

def _initialize_currents():
    """Initial currents are set to the first distorted pattern."""

    currents = np.copy(distort_pattern(
        patterns[self.first_p],
        inverted_fraction)
    )

    # print("\nInitial currents:\n", self.currents)

def calculate_next_weights(pattern, next_theoretical_weights=next_theoretical_weights):
    """
    Calculate the weights after the presentation of a new pattern but does
    not change the current weights of the network
    """
    for i in tqdm(range(num_neurons)):
        for j in range(num_neurons):
            if j >= i:
                break

            next_theoretical_weights[i, j] += (2 * pattern[i] - 1) \
                                                   * (2 * pattern[j] - 1) \

    next_theoretical_weights += next_theoretical_weights.T

def update_weights(weights):
    weights += weights

    def compute_weights_all_patterns():
        print(f"\n...Computing weights for all patterns...\n")

        for p in tqdm(range(len(patterns))):
            calculate_next_weights(patterns[p])
            update_weights(next_theoretical_weights)
            print(next_theoretical_weights)

        print("Done!")


def _activation_function(x):
    """Heaviside"""
    return int(x >= 0)

def noise():
    # Amplitude-modulated Gaussian noise
    return np.random.normal(loc=0, scale=noise_variance**0.5) * 0.05

def _update_current(neuron):
    """
    If you are updating one node of a Hopfield network, then the values of
    all the other nodes are input values, and the weights from those nodes
    to the updated node as the weights.

    In other words, first you do a weighted sum of the inputs from the
    other nodes, then if that value is greater than or equal to 0, you
    output 1. Otherwise, you output 0

    :param neuron: int neuron number
    """
    dot_product = np.dot(weights[neuron], currents[-2])

    currents[-1, neuron] = _activation_function(dot_product
                                                          + noise())

def update_all_neurons(currents=currents):
    """
    Neurons are updated update in random order as described by Hopfield.
    The full network should be updated before the same node gets updated
    again.
    """
    # self.last_currents = self.currents

    values = np.arange(0, num_neurons, 1)
    update_order = np.random.choice(values,
                                    num_neurons,
                                    replace=False)

    currents = np.vstack((currents, np.zeros(num_neurons)))

    for i in update_order:
        _update_current(i)

    # self.currents_history = np.vstack((self.currents_history,
    #                                    self.currents))

def _update_current_learning(neuron):
    """
    If you are updating one node of a Hopfield network, then the values of
    all the other nodes are input values, and the weights from those nodes
    to the updated node as the weights.

    In other words, first you do a weighted sum of the inputs from the
    other nodes, then if that value is greater than or equal to 0, you
    output 1. Otherwise, you output 0

    :param neuron: int neuron number
    """
    random_currents = \
        np.random.choice([0, 1], p=[1 - self.f, self.f],
                         size=num_neurons)
    dot_product = np.dot(weights[neuron], random_currents)

    self.currents[-1, neuron] = _activation_function(dot_product
                                                          + noise())

def update_all_neurons_learning():
    """
    Neurons are updated update in random order as described by Hopfield.
    The full network should be updated before the same node gets updated
    again.
    """
    # self.last_currents = self.currents

    values = np.arange(0, num_neurons, 1)
    neuron_update_order = np.random.choice(values,
                                           num_neurons,
                                           replace=False)

    currents = np.vstack((currents, np.zeros(num_neurons)))

    for neuron in neuron_update_order:
        self._update_current(neuron)

    # self.currents_history = np.vstack((self.currents_history,
    #                                    self.currents))

def _compute_patterns_evolution():

    for p in range(p):
        similarity = np.sum(currents[-1] == patterns[p])
        self.patterns_evolution = \
            np.vstack((patterns_evolution, similarity))

    patterns_evolution = patterns_evolution.T
    patterns_evolution = patterns_evolution[0, 1:]

def _find_attractor():
    """
    If an update does not change any of the node values, the network
    rests at an attractor and updating stops.
    """
    tot = 1

    while (currents[-1] != currents[-2]).all() or tot < 2:  # np.sum(self.currents - self.last_currents) != 0:
        self.update_all_neurons()
        self._compute_patterns_evolution()
        tot += 1
        print(f"\nUpdate {tot} finished.\n")

    attractor = np.int_(np.copy(currents[-1]))

    print(f"\nFinished as attractor {attractor} after {tot} "
          f"node value updates.\n")

def simulate():
    # assert patterns
    # assert num_neurons == patterns[0].size

    # _initialize()
    compute_weights_all_patterns()
    _initialize_currents()
    update_all_neurons()
    _find_attractor()

def learn(item=None, time=None):
    """
    The normalized difference of means calculated at every time step gives
    a logarithmic emergent behavior as the weights get closer to the
    theoretical ones.

    :param item:
    :param time:
    :return:
    """

    next_weights = (next_theoretical_weights - weights) \
        * learning_rate
    # print(self.next_weights)

    update_weights(next_weights)

    weights_mean.append(-np.mean(next_theoretical_weights)
                             + np.mean(weights))

    # plot.attractor_networks.plot_weights(self)
    # pass

def fully_learn(self):
    # tot = 1
    #
    # while (self.weights[-1] != self.next_theoretical_weights).all():
    #     self.learn()
    #     tot += 1
    #
    # print(f"\nFinished learning after {tot} "
    #       f"node weight updates.\n")
    pass

def forget():
    next_weights = (weights + noise() * 10000000) \
        * forgetting_rate

    update_weights(next_weights)

    weights_mean.append(-np.mean(next_theoretical_weights)
                             + np.mean(weights))

def simulate_learning(iterations, recalled_pattern):

    if p == 1:
        calculate_next_weights(patterns[first_p])
        update_all_neurons()
    else:
        # for p in range(self.p - 2):
        for p in range(len(patterns - 1)):
            calculate_next_weights(patterns[p])
            update_weights(next_theoretical_weights)

        # self.currents = np.vstack((self.currents,
                                   # np.zeros(self.num_neurons)))
        calculate_next_weights(patterns[self.p - 1])
        update_all_neurons()
    # self.update_all_neurons()

    p_recall(n_pattern=recalled_pattern)

    for i in range(iterations):
        learn()
        update_all_neurons()
        p_recall(n_pattern=recalled_pattern)

def p_recall(item=None, n_pattern=None, time=None, verbose=False):
    """
    After choosing, compare the chosen pattern with the correct pattern
    to retrieve the probability of recall.
    """
    assert item is not None or n_pattern is not None
    if item is not None:
        bin_item = binarize_item(item)
    if n_pattern is not None:
        bin_item = patterns[n_pattern]

    # self.currents = np.vstack((self.currents, bin_item))
    # self.update_all_neurons()

    match = np.sum(currents[-1] == bin_item)
    p_r = match / num_neurons
    p_recall_history.append(p_r)

    if verbose:
        print("\nCurrent after item presentation and one update:\n",
              currents[-1])
        print("\nProbability of recall of the item: ", p_r)

    return p_r

# End of original class methods #


def main(force=False, next_theoretical_weights=next_theoretical_weights):

    bkp_file = f"bkp/hopfield.p"

    os.makedirs(os.path.dirname(bkp_file), exist_ok=True)

    if not os.path.exists(bkp_file) or force:

        np.random.seed(123)

        # flower = {"kanji": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        #           "meaning": np.array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1])}
        #
        # leg = {"kanji": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
        #        "meaning": np.array([0, 0, 0, 0, 1, 1, 0, 1, 0, 1])}
        #
        # eye = {"kanji": np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
        #        "meaning": np.array([0, 0, 0, 0, 1, 0, 1, 1, 1, 1])}


        num_neurons=80,
        f=0.55,
        p=2,
        first_p=0,
        inverted_fraction=0.5,
        learning_rate=0.1,
        forgetting_rate=0.1


        # network.present_pattern(flower)
        # network.present_pattern(leg)
        # network.present_pattern(eye)

        # # learning loop
        # network.calculate_next_weights(network.patterns[0])
        # network.update_weights(network.next_theoretical_weights)
        # network.calculate_next_weights(network.patterns[0])
        # network.update_all_neurons()
        #
        # network.p_recall(n_pattern=0)
        #
        # for i in range(20):
        #     network.learn()
        #     network.update_all_neurons_learning()
        #     network.p_recall(n_pattern=0)

        calculate_next_weights(patterns[0])
        update_weights(next_theoretical_weights)
        update_all_neurons()
        plot_weights()
        calculate_next_weights(patterns[1])
        p_recall(n_pattern=1)

        print(next_theoretical_weights)

        for i in range(1750):
            learn()
            update_all_neurons_learning()
            p_recall(n_pattern=1)

        plot_weights(network)

        # network.simulate_learning(iterations=100, recalled_pattern=2)

        weights = np.zeros_like(next_theoretical_weights)
        next_theoretical_weights = np.zeros_like(weights)
        print(weights)
        compute_weights_all_patterns()
        plot_weights()


        pickle.dump(open(bkp_file, "wb"))
    else:
        print("Loading from pickle file...")
        network = pickle.load(open(bkp_file, "rb"))

    plot_mean_weights()
    plot_energy()
    plot_p_recall()
    plot_currents()
    # plot_weights()


if __name__ == '__main__':
    main(force=True)


 30%|██▉       | 299/1000 [00:00<00:00, 2987.18it/s]


Patterns:
 [[0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


100%|██████████| 1000/1000 [00:01<00:00, 915.89it/s]


IndexError: index -2 is out of bounds for axis 0 with size 1